In [1]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os


In [2]:
# Combine all alphas
def get_stat():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            # print(fileName)
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            # print(df)
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res


In [3]:
stats = get_stat()

In [4]:
stats.describe()

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.112360,-0.092363,1.610674,-0.701595,0.009686,0.112088,0.379563,0.114387
std,0.107743,0.038771,1.251950,0.318042,0.007817,0.095287,0.082778,0.115912
min,-0.077258,-0.203672,-0.874406,-1.549496,-0.004113,0.020960,0.258523,0.010282
25%,0.035184,-0.117383,0.760680,-0.888662,0.002178,0.057449,0.302470,0.033663
50%,0.122559,-0.091967,2.102997,-0.720338,0.010778,0.063902,0.376716,0.068256
75%,0.195175,-0.058211,2.592515,-0.371849,0.016176,0.128797,0.438348,0.137830
max,0.285464,-0.038737,3.208175,-0.285176,0.021537,0.350836,0.589292,0.748058


In [5]:
good_alphas = list(stats.loc[stats.IR > 2].index)

In [7]:
# combine daily pnl
def get_pnl():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            df = pd.read_csv(file)
            # df = df.set_index('time')
            # df = df['pnl']
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res


In [8]:
pnls = get_pnl()

In [9]:
corr = pnls[good_alphas].corr()

In [42]:
from itertools import combinations
import random
res = []
num = 5
good_candidates = list(
    ((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
random.shuffle(good_candidates)
for cols in combinations(good_candidates, num):
    corr_small = corr.loc[cols, cols]
    if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
        res.append(corr_small)

In [43]:
max_IR = float('-inf')
max_idx = -1
for i in range(len(res)):
    total_IR = np.sum([(stats.loc[stats.index==name].IR) for name in res[i].index])
    if total_IR > max_IR:
        max_IR = total_IR
        max_idx = i

In [44]:
res[max_idx]

IndexError: list index out of range

In [45]:
res

[]